In [1]:
# import realted module
import pandas as pd
import numpy as np

# read the file
leave16= pd.read_csv('F:/leave16.csv',header=None, delimiter=',', 
                     names = ['dayofservice','tripid','progrnumber','stoppointid','plannedtime_arr','plannedtime_dep','actualtime_arr','actualtime_dep','suppressed'])

In [10]:
# Check the rows and columns number of the table
leave16.shape

(56843319, 9)

In [11]:
# Get an overview of the table
leave16.head()

,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed
0,2016-01-31,2811044,63,3390,41820,41820,42920,42920,NaN
1,2016-01-31,2820852,1,7158,69300,69300,69320,69320,NaN
2,2016-01-31,2820852,5,7017,69498,69498,69532,69532,NaN
3,2016-01-31,2820852,10,1893,69711,69711,69858,69858,NaN
4,2016-01-31,2820853,16,1648,76822,76822,76636,76657,NaN


In [12]:
# Read the trips file
trips16 = pd.read_csv('F:/trips16.csv', delimiter=',')

# Check the shape of the table
trips16.shape

(928098, 10)

In [13]:
# Get an overview of the trips table
trips16.head()

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed
0,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN
1,2016-02-09,2826717,9,9_7,2,65553,60600,66258.0,60582.0,NaN
2,2016-02-09,2826730,54A,54A_12,2,65951,62100,66053.0,62078.0,NaN
3,2016-02-09,2826743,7,7_51,1,54763,50400,54854.0,50383.0,NaN
4,2016-02-09,2812908,39,39_20,1,27375,22920,27318.0,22931.0,NaN


In [14]:
# Read the weather data
weather16 = pd.read_csv('2016weatherClean.csv', delimiter=',')
# weather16['dt'] = pd.to_datetime(weather16['dt'],unit='s')
weather16 = weather16.sort_values('dt', ascending=False)
weather16 = weather16.reindex(index=weather16.index[::-1])

Here we won't add all features of the weather data to our model. We just select some important ones, including the followings:
- temp
- pressure
- humidity
- wind_speed
- weather_description
- clouds_all
- wind_deg

In [15]:
weather_related = weather16[['dt','temp','pressure','humidity','wind_speed','weather_description','clouds_all','wind_deg']]
weather_related.head()

,dt,temp,pressure,humidity,wind_speed,weather_description,clouds_all,wind_deg
0,1451606400,275.15,1015,80,4,scattered clouds,40,200
1,1451613600,274.15,1016,80,2,scattered clouds,40,170
2,1451620800,276.15,1015,69,3,scattered clouds,40,130
3,1451628000,277.09,1014,69,4,scattered clouds,40,150
4,1451631600,277.09,1013,100,3,broken clouds,75,150


In [16]:
# Select rows with lineid == 46A and direction == 1 
trips16_46A = trips16.loc[(trips16['lineid'] == '46A') & (trips16['direction'] == 1)]
trips16_46A.head(10)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed
103,2016-02-09,2815441,46A,46A_74,1,62257,57600,62587.0,57639.0,NaN
154,2016-02-09,2822373,46A,46A_74,1,33599,28320,33931.0,28323.0,NaN
199,2016-04-08,3010331,46A,46A_74,1,77695,74400,77565.0,74375.0,NaN
223,2016-04-02,2902305,46A,46A_74,1,66140,61800,66544.0,62003.0,NaN
304,2016-04-02,2905183,46A,46A_74,1,53647,49200,53670.0,49630.0,NaN
330,2016-04-02,2909721,46A,46A_74,1,51847,47400,51699.0,47370.0,NaN
366,2016-04-07,3018107,46A,46A_74,1,61906,57120,62097.0,56944.0,NaN
367,2016-04-07,3014232,46A,46A_74,1,60946,56160,61634.0,56200.0,NaN
368,2016-04-07,3004076,46A,46A_74,1,63346,58560,63126.0,58529.0,NaN
369,2016-04-07,3008876,46A,46A_74,1,62386,57600,62098.0,57579.0,NaN


In [17]:
# Join the trip table and leavetime table, use inner jopz
out = pd.merge(trips16_46A , leave16, on = ['tripid', 'dayofservice'], how = 'inner')

In [18]:
out.shape

(889694, 17)

In [11]:
out.head(1)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr_x,plannedtime_dep_x,actualtime_arr_x,actualtime_dep_x,suppressed_x,progrnumber,stoppointid,plannedtime_arr_y,plannedtime_dep_y,actualtime_arr_y,actualtime_dep_y,suppressed_y
0,2016-02-09,2815441,46A,46A_74,1,62257,57600,62587.0,57639.0,NaN,59,2039,62257,62257,62587,62587,NaN


Features:
- start_stop
- end_stop
- date
- weather

Target Value:
- Time. (The dearture time of end - The departure time of start)

Notes: The start_stop and end_stop is adjancent.

In [36]:
trip_46A = out['tripid'].unique()
date_46A = out['dayofservice'].unique()

result = pd.DataFrame(columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration'])

for trip in trip_46A:
    for date in date_46A:
        
        day_trip = out.loc[(out['tripid'] == trip) & (out['dayofservice'] == date)].sort_values(by=['progrnumber'])
        
        if not day_trip.empty:
            
            try:
                day_trip.index = range(len(day_trip))

                # Calculate the time between each pair stops
                time = day_trip['actualtime_arr_y'].shift(-1) - day_trip['actualtime_arr_y']
                time.drop(time.tail(1).index,inplace=True)

                # Get the month and the day, month and the day of week
                month = pd.to_datetime(day_trip['dayofservice']).dt.month
                day = pd.to_datetime(day_trip['dayofservice']).dt.day
                dayofweek = pd.to_datetime(day_trip['dayofservice']).dt.dayofweek

                # Add datetime and actual_arrive time as current unix time
                datetime = pd.DatetimeIndex(day_trip['dayofservice']).astype(np.int64)/1000000000 + day_trip['actualtime_arr_y']
                datetime = pd.DataFrame(data={'unixtime':datetime})

                # Conver float to int, this is for the following merge operation
                datetime['unixtime'] = datetime['unixtime'].astype(np.int64)

                # Set End point 
                end_point = day_trip['stoppointid'].shift(-1)
                end_point.drop(end_point.tail(1).index,inplace=True)

                # Merge these columns
                a = pd.concat([datetime,dayofweek, month, day, day_trip['actualtime_arr_y'], day_trip['stoppointid'], end_point, time], axis = 1)

                # Change the name of columns
                a.columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration']
                a.drop(a.tail(1).index,inplace=True)

                # Merge two tables
                r = pd.merge_asof(a, weather_related, on = "dt",direction='nearest')

                result = pd.concat([r, result])
                
            except:
                pass
        
# result

KeyboardInterrupt: 

In [38]:
result.shape
result

,arrive_time,clouds_all,day,dayofweek,dt,duration,end_point,humidity,month,pressure,start_point,temp,weather_description,wind_deg,wind_speed
0,37578,0.0,18,5,1466245578,33.0,808.0,62.0,6,1020.0,807,287.04,Sky is Clear,290.0,6.0
1,37611,0.0,18,5,1466245611,20.0,809.0,62.0,6,1020.0,808,287.04,Sky is Clear,290.0,6.0
2,37631,0.0,18,5,1466245631,67.0,810.0,62.0,6,1020.0,809,287.04,Sky is Clear,290.0,6.0
3,37698,0.0,18,5,1466245698,43.0,811.0,62.0,6,1020.0,810,287.04,Sky is Clear,290.0,6.0
4,37741,0.0,18,5,1466245741,51.0,812.0,62.0,6,1020.0,811,287.04,Sky is Clear,290.0,6.0
5,37792,0.0,18,5,1466245792,24.0,813.0,62.0,6,1020.0,812,287.04,Sky is Clear,290.0,6.0
6,37816,24.0,18,5,1466245816,49.0,814.0,62.0,6,1014.0,813,287.04,few clouds,270.0,5.0
7,37865,24.0,18,5,1466245865,113.0,81.0,62.0,6,1014.0,814,287.04,few clouds,270.0,5.0
8,37978,24.0,18,5,1466245978,64.0,817.0,62.0,6,1014.0,81,287.04,few clouds,270.0,5.0
9,38042,24.0,18,5,1466246042,14.0,818.0,62.0,6,1014.0,817,287.04,few clouds,270.0,5.0


In [40]:
result.to_csv('part_result_of_46A.csv')